# MNIST Data Loader

In [22]:
# %load mnist_loader.py
"""
mnist_loader
~~~~~~~~~~~~~~~
A library to load the MNIST image data. For details of the data 
structures that are returned, see the doc strings for 'load_data'
and 'load_data_wrapper'. In practice, 'load_data_wrapper' is the
function usually called by our neural network code."""

### Libraries
# Standard library
import pickle
import gzip

    # Third-party Libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    The 'training_data' is returned as a tuple with two entries.
    The first entry contains the actural training images. This is a
    numpy ndarray with 50,000 entries. each entry is, in turn, a 
    numpy ndarray with 784 values. representing the 28*28 = 784
    pixels in a single MNIST image.
    The second entry in the 'training_data' tuple is a numpy ndarray
    containing 50,000 entries. Those entries are just the digit values
    (0, 1...,9) for the corresponding images containded in the first
    entry of the tuple.
    The 'validation_data' and 'test_data' are similar, except each contains
    only 10,000 images.
    This is a nice data format, but for use in neural networks it's 
    helpful to modify the format of the 'training_data' a little.
    That's done in the wrapper function 'load_data_wrapper()', see below."""
    
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)


def load_data_wrapper():
    """Return a tuple containing '(training_data, validation_data,
    test_data)'. Based on 'load_data', but the format is more convenient
    for use in our implementation of neural networks. In particular, 
    'training_data' is a list containing 50,000 2-tuples '(x, y)'. 'x' is
    a 784-dimensional numpy.ndarray representing the unit vector corresponding
    to the correct digit for 'x'.
    
    'validation_data' and 'test_data' are lists containing 10,000
    2-tuples '(x, y)'. In each case, 'x' is a 784-dimensional
    numpy.ndarray containing the input image, and 'y' is the corresponding
    classification , i.e., the digit values (integers) corresponding
    to 'x'.
    
    Obviously, this means we're using slightly different formats for
    the training data and the validation/test data. These formats
    turn out to be the most convenient for use in our neural network code."""
    
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)


def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeros elsewhere. This is used to convert a digit
    (0,....,9) into a corresponding desired output from the neural network."""
    
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


##############################################################################################################################

In [ ]:
import random
import numpy as np

class Network(object):
    
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the Output of the network if 'a' is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
            """Train the neural network using mini-batch stochastic gradient descent. The training_data is a list of 
            tuples(x, y) representing the training inputs and the desired outputs. The other non-optional parameters 
            are self-explanatory. If test_data is provided then the network will be evaluated against the test data 
            after each epoch, and partial progress printed out. This is useful for tracking progress, but slows 
            things down substantially."""
            training_data = list(training_data)
            n = len(training_data)

            if test_data: 
                test_data = list(test_data)
                n_test = len(test_data)
            
            for j in range(epochs):
                random.shuffle(training_data)
                mini_batches = [training_data[k: k+mini_batch_size] for k in range(0, n, mini_batch_size)]
                for mini_batch in mini_batches:
                    self.update_mini_batch(mini_batch, eta)
                if test_data:
                    print(f"Epoch {j}:{self.evaluate(test_data)} / {n_test}")
                else:
                    print(f"Epoch {j} complete")

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying gradient descent
        using backpropagation to a single mini batch. The 'mini_batch' is a list
        of 'tuples (x, y)'', and 'eta' is the learning rate. """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """return a tuple '(nabla_b, nabla_w)' representing the gradient for 
        the cost function C_x. 'nabla_b' and 'nabla_w' are layer-by-layer lists
        of numpy arrays, similar to 'self.biases' and 'self.weights'."""

        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
    #feed_forward
        activation = x
        activations = [x] # List to store all the activations, layer by layer
        zs = [] # List to stores all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
    # Backward Pass
        delta = self.cost_derivative(activations[-1], y)* sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    # Note that the variable l in the loop below is used a little 
    # Diffrently to the notation in Chapter-2 of the book. Here,
    # l = 1 means the last layer of neurons, l = 2 is the second-last layer and so on.
    # It's a renumbering of the scheme in the book, used here to take advantage
    # of the fact that python can use negative indices in lists.

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta)*sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l+1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the nubmer of test inputs for which the neural network outputs
        the correct result. Note that the neural network's output is assumed to
        be the index of whichever neuron in the final layer has the highest activation."""

        test_results = [(np.argmax(self.feedforward(x)),y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives/partial C_x /partial a for
        the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [5]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [6]:
import network
net = network.Network([784, 30, 10])

In [3]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0 : 9096 / 10000
Epoch 1 : 9246 / 10000
Epoch 2 : 9225 / 10000
Epoch 3 : 9289 / 10000
Epoch 4 : 9301 / 10000
Epoch 5 : 9358 / 10000
Epoch 6 : 9341 / 10000
Epoch 7 : 9382 / 10000
Epoch 8 : 9366 / 10000
Epoch 9 : 9383 / 10000
Epoch 10 : 9414 / 10000
Epoch 11 : 9400 / 10000
Epoch 12 : 9397 / 10000
Epoch 13 : 9441 / 10000
Epoch 14 : 9456 / 10000
Epoch 15 : 9444 / 10000
Epoch 16 : 9428 / 10000
Epoch 17 : 9439 / 10000
Epoch 18 : 9483 / 10000
Epoch 19 : 9473 / 10000
Epoch 20 : 9479 / 10000
Epoch 21 : 9475 / 10000
Epoch 22 : 9467 / 10000
Epoch 23 : 9472 / 10000
Epoch 24 : 9458 / 10000
Epoch 25 : 9457 / 10000
Epoch 26 : 9455 / 10000
Epoch 27 : 9474 / 10000
Epoch 28 : 9462 / 10000
Epoch 29 : 9468 / 10000


In [7]:
net.SGD(training_data, 30, 10, 1.0, test_data = test_data)

Epoch 0 : 8816 / 10000
Epoch 1 : 9057 / 10000
Epoch 2 : 9151 / 10000
Epoch 3 : 9191 / 10000
Epoch 4 : 9263 / 10000
Epoch 5 : 9282 / 10000
Epoch 6 : 9312 / 10000
Epoch 7 : 9331 / 10000
Epoch 8 : 9339 / 10000
Epoch 9 : 9374 / 10000
Epoch 10 : 9365 / 10000
Epoch 11 : 9377 / 10000
Epoch 12 : 9396 / 10000
Epoch 13 : 9392 / 10000
Epoch 14 : 9417 / 10000
Epoch 15 : 9403 / 10000
Epoch 16 : 9419 / 10000
Epoch 17 : 9415 / 10000
Epoch 18 : 9405 / 10000
Epoch 19 : 9420 / 10000
Epoch 20 : 9445 / 10000
Epoch 21 : 9441 / 10000
Epoch 22 : 9457 / 10000
Epoch 23 : 9450 / 10000
Epoch 24 : 9441 / 10000
Epoch 25 : 9453 / 10000
Epoch 26 : 9448 / 10000
Epoch 27 : 9453 / 10000
Epoch 28 : 9462 / 10000
Epoch 29 : 9458 / 10000


In [8]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

import network
net = network.Network([784, 50, 10])

In [9]:
net.SGD(training_data, 30, 10, 1.0, test_data = test_data)

Epoch 0 : 8821 / 10000
Epoch 1 : 9122 / 10000
Epoch 2 : 9254 / 10000
Epoch 3 : 9297 / 10000
Epoch 4 : 9350 / 10000
Epoch 5 : 9357 / 10000
Epoch 6 : 9366 / 10000
Epoch 7 : 9399 / 10000
Epoch 8 : 9414 / 10000
Epoch 9 : 9389 / 10000
Epoch 10 : 9432 / 10000
Epoch 11 : 9441 / 10000
Epoch 12 : 9449 / 10000
Epoch 13 : 9463 / 10000
Epoch 14 : 9457 / 10000
Epoch 15 : 9442 / 10000
Epoch 16 : 9462 / 10000
Epoch 17 : 9471 / 10000
Epoch 18 : 9469 / 10000
Epoch 19 : 9477 / 10000
Epoch 20 : 9482 / 10000
Epoch 21 : 9485 / 10000
Epoch 22 : 9482 / 10000
Epoch 23 : 9493 / 10000
Epoch 24 : 9494 / 10000
Epoch 25 : 9507 / 10000
Epoch 26 : 9503 / 10000
Epoch 27 : 9514 / 10000
Epoch 28 : 9506 / 10000
Epoch 29 : 9506 / 10000


In [10]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

import network
net = network.Network([784, 30, 20, 10])

In [11]:
net.SGD(training_data, 30, 10, 1.0, test_data = test_data)

Epoch 0 : 8652 / 10000
Epoch 1 : 9015 / 10000
Epoch 2 : 9153 / 10000
Epoch 3 : 9227 / 10000
Epoch 4 : 9235 / 10000
Epoch 5 : 9304 / 10000
Epoch 6 : 9345 / 10000
Epoch 7 : 9384 / 10000
Epoch 8 : 9396 / 10000
Epoch 9 : 9407 / 10000
Epoch 10 : 9391 / 10000
Epoch 11 : 9435 / 10000
Epoch 12 : 9440 / 10000
Epoch 13 : 9404 / 10000
Epoch 14 : 9423 / 10000
Epoch 15 : 9425 / 10000
Epoch 16 : 9437 / 10000
Epoch 17 : 9455 / 10000
Epoch 18 : 9446 / 10000
Epoch 19 : 9480 / 10000
Epoch 20 : 9455 / 10000
Epoch 21 : 9473 / 10000
Epoch 22 : 9474 / 10000
Epoch 23 : 9476 / 10000
Epoch 24 : 9474 / 10000
Epoch 25 : 9478 / 10000
Epoch 26 : 9485 / 10000
Epoch 27 : 9488 / 10000
Epoch 28 : 9478 / 10000
Epoch 29 : 9453 / 10000


In [12]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

import network
net = network.Network([784, 10])

In [13]:
net.SGD(training_data, 30, 10, 1.0, test_data = test_data)

Epoch 0 : 5661 / 10000
Epoch 1 : 6545 / 10000
Epoch 2 : 6601 / 10000
Epoch 3 : 6613 / 10000
Epoch 4 : 6630 / 10000
Epoch 5 : 6628 / 10000
Epoch 6 : 6623 / 10000
Epoch 7 : 6646 / 10000
Epoch 8 : 6683 / 10000
Epoch 9 : 7377 / 10000
Epoch 10 : 7429 / 10000
Epoch 11 : 7495 / 10000
Epoch 12 : 7774 / 10000
Epoch 13 : 8336 / 10000
Epoch 14 : 8363 / 10000
Epoch 15 : 8347 / 10000
Epoch 16 : 8382 / 10000
Epoch 17 : 8374 / 10000
Epoch 18 : 8396 / 10000
Epoch 19 : 8397 / 10000
Epoch 20 : 8402 / 10000
Epoch 21 : 8414 / 10000
Epoch 22 : 8393 / 10000
Epoch 23 : 8408 / 10000
Epoch 24 : 8390 / 10000
Epoch 25 : 8426 / 10000
Epoch 26 : 8415 / 10000
Epoch 27 : 8427 / 10000
Epoch 28 : 8419 / 10000
Epoch 29 : 8392 / 10000
